<a href="https://colab.research.google.com/github/Strojove-uceni/23206-final-sign-language-recognition-in-video/blob/main/NN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import math
import sys
import cv2
import io

from google.colab import drive

In [2]:
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Colab Notebooks/SU2/data"

Mounted at /content/drive


In [3]:
!git clone https://github.com/jiri99/SU_sign-language.git

Cloning into 'SU_sign-language'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14 (delta 5), reused 8 (delta 2), pack-reused 0
Receiving objects: 100% (14/14), 6.44 KiB | 6.44 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [4]:
repo_path = "/content/SU_sign-language/"
sys.path.append(os.path.abspath(repo_path))

In [5]:
from buffer import ParquetData

In [ ]:
selected_landmark_indices = [33, 133, 159, 263, 46, 70, 4, 454, 234, 10, 338, 297, 332, 61, 291, 0, 78, 14, 317,
                             152, 155, 337, 299, 333, 69, 104, 68, 398]

df_train = pd.read_csv(path + "/train_mod.csv", sep=",")
df_train.head()

data_load = ParquetData()
data_load.read_all(path, df_train, selected_landmark_indices)

In [7]:
data_load.data["water"]["2044"].tensor.shape

(70, 70, 420)

In [43]:
stacked_data = []
stacked_sign = []

for key_sign in list(data_load.data.keys()):
  for key_participiant in list(data_load.data[key_sign].keys()):
    stacked_sign.append(key_sign)
    stacked_data.append(data_load.data[key_sign][key_participiant].tensor)

tensor_data = np.stack(stacked_data, axis=0)
X_train = tensor_data.reshape((len(stacked_sign), 420, 70, 70, 1))
stack_sign = np.stack(stacked_sign, axis=0)
one_hot_encoded = pd.get_dummies(stack_sign)
Y_train = tf.convert_to_tensor(one_hot_encoded, dtype=tf.float32)

In [44]:
X_train.shape

(252, 420, 70, 70, 1)

In [45]:
Y_train.shape

TensorShape([252, 250])

In [49]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed

frame_height = 70
frame_width = 70
channels = 1
classes_num = len(list(data_load.data.keys()))
sequence_length = 420

# CNN model for spatial feature extraction
cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(frame_height, frame_width, channels)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten()
])

# Final model
model = Sequential([
    TimeDistributed(cnn, input_shape=(sequence_length, frame_height, frame_width, channels)),
    LSTM(64, return_sequences=False),
    Dense(64, activation='relu'),
    Dense(classes_num, activation='softmax')  # Assuming 10 classes for the classification
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_4 (TimeDi  (None, 420, 6272)         92672     
 stributed)                                                      
                                                                 
 lstm_4 (LSTM)               (None, 64)                1622272   
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                                 
 dense_9 (Dense)             (None, 250)               16250     
                                                                 
Total params: 1735354 (6.62 MB)
Trainable params: 1735354 (6.62 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [50]:
# Now you can fit this model on your preprocessed data.
# X_train would be your video data with shape (num_samples, sequence_length, frame_height, frame_width, channels)
# Y_train would be your labels
model.fit(X_train, Y_train, epochs=10, batch_size=4)

Epoch 1/10


ResourceExhaustedError: ignored